# Image SRK example

This example shows how to create fuses file (SRK) from certificates.


In [1]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-
#
# Copyright 2019-2023 NXP
#
# SPDX-License-Identifier: BSD-3-Clause

%run ../init_notebook.ipynb

import os
import sys

from spsdk.crypto.certificate import Certificate
from spsdk.image.secret import SrkItem, SrkTable
from spsdk.utils.misc import Endianness, write_file

# The path to directory with certificates
DATA_DIR = os.path.join("..", "_data", "certificates")


# Gather certificate files from data directory
cert_files = [f for f in os.listdir(DATA_DIR) if f.endswith(".pem")]
if not cert_files:
    print(f'Add generated *.pem files into "{DATA_DIR}" directory')
    sys.exit()

# Create SRK Table instance
srk_table = SrkTable(version=0x40)

for cert_file in cert_files:
    certificate = Certificate.load(f"{DATA_DIR}/{cert_file}")
    srk_item = SrkItem.from_certificate(certificate)
    srk_table.append(srk_item)

print(srk_table)

# Export fuses
write_file(srk_table.export_fuses(), f"{DATA_DIR}/srk_fuses.bin", "wb")
# Export binary SRK table
write_file(srk_table.export(), f"{DATA_DIR}/srk_table.bin", "wb")

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.
------------------------------------------------------------
SRK Table (Version: 4.0, #Keys: 1)
------------------------------------------------------------
SRK Key Index: 0 
Algorithm: EnumAlgorithm.PKCS1
Flag:      0x00 
Length:    2048 bit
Modulus:
    00:a6:5b:76:d8:ec:47:24:be:b3:78:57:c5:bf:76:
    47:b4:e9:19:98:62:09:9c:ad:39:70:b1:e4:82:13:
    36:89:25:54:41:a0:2b:c5:58:de:5d:7c:08:a8:3a:
    78:ab:42:d2:f4:bc:da:0e:24:d2:d1:8d:54:d2:5f:
    af:41:7a:a1:92:f3:12:85:65:c1:d6:70:f4:ec:31:
    c9:95:9c:7f:60:86:c0:06:4f:2e:b9:fa:be:fe:d6:
    3e:d4:c7:82:e6:60:2a:8e:e3:b3:8a:b4:a4:8d:8f:
    07:a0:37:72:cd:e2:9b:cf:f6:90:c1:fd:07:3b:cf:
    d0:fd:f4:0d:4a:20:db:fc:2e:51:1a:29:2e:41:7c:
    50:a3:77:ec:10:8a:88:aa:05:fb:d4:e1:e9:3b:e3:
    0e:39:c5:67:45:2a:30:34:a5:13:43:09:3e:9b:5b:
    cb:38:8b:57:be:31:ed:6b:c2:fe:72:6b:22:1c:0f:
    6c:be:18:9a:70:87:b2:45:98:7f:cf:92:25:54:d4:
    32:eb:df:11:a5:90:64:7a:03:33:2c

275

In [2]:
# The path to directory with keys
DATA_DIR = os.path.join("..", "_data", "keys", "ecc521")
keys = ["srk0_ecc521.pem", "srk1_ecc521.pem", "srk2_ecc521.pem", "srk3_ecc521.pem"]
# AHAB SRK Table

from spsdk.crypto.utils import extract_public_key
from spsdk.image.ahab.ahab_container import SRKTable as AhabSrkTable

ahab_srk = AhabSrkTable()

for key in keys:
    ahab_srk.add_record(extract_public_key(f"{DATA_DIR}/{key}"))

ahab_srk.update_fields()
ahab_srk_hash = ahab_srk.compute_srk_hash()

print(ahab_srk)

# Export binary SRK table
print("SRK table in binary:")
print(ahab_srk.export().hex())

# Display SRK hash fuses values
print("SRK hash fuses values:")
for i in range(0, len(ahab_srk_hash), 4):
    print("0x%X" % int.from_bytes(ahab_srk_hash[i : i + 4], byteorder=Endianness.LITTLE.value))

AHAB SRK table:
  Keys count:         4
  Length:             580
SRK table HASH:       53471a1098ae0ad84ac8e192f31864a5121f8963a512c9ef37ea1cb9a7a8f83a
SRK table in binary:
d7440242e1900027020300004200420000c51e858b17a8f03d637807a6085d49cf41af200ca6ac79939489528db75f4268d669f1360de5fe8bc5d8d95773221629d958dfd34528dd6ba632a7f7e616cf1e2f0101a745a331759c546ede791e80fb6231fe512e3204d8021b25be0f08d6076d8c1c56740b5eb5d0166f2219822242d66f83de347337a33fb1f55d6a690bc2651ec1e19000270203000042004200015f79416d4966e05f048d0c596ffdbf5bc763d1781ea981ac4ab067fd336788fe1ac8daa7857ad692b570d47b3a7de2c206d04f3e1d1eaece635bd4737ff0ae54aa007a8e94a15dc8aecf4af5cdc4eb4013befd262f907908ab89cb7e8bd4baf5124282533a729f66ff35f82aaccdbc74766bdb8f3a0356353efe5c24d307e0661f8d29e1900027020300004200420001e0e61acb896436c79c76235ce89a736b460763eb1bf0f039daf2f453ea1b1a32aca8e0753924f3b26948ff49c26a1183a09537f40ddf63a244e79becf523d78c9b003d224c361c0e41f362b33386067f7c7b2fd873fb799cbcf52702d763c0eee184823616865ab1adf1bc3f